# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298502834027                   -0.85    5.0
  2   -8.300201796753       -2.77       -1.25    1.2
  3   -8.300446555879       -3.61       -1.89    4.0
  4   -8.300461852354       -4.82       -2.77    1.4
  5   -8.300464310459       -5.61       -3.06    3.4
  6   -8.300464463608       -6.81       -3.23    1.0
  7   -8.300464551883       -7.05       -3.37    1.5
  8   -8.300464596304       -7.35       -3.50    1.4
  9   -8.300464630005       -7.47       -3.68    1.1
 10   -8.300464639045       -8.04       -3.83    1.1
 11   -8.300464643757       -8.33       -4.31    1.0


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.63686903984                   -0.71    5.2
  2   -16.67745702619       -1.39       -1.14    1.4
  3   -16.67898793206       -2.82       -1.87    4.4
  4   -16.67919464231       -3.68       -2.51    4.5
  5   -16.67926885166       -4.13       -3.03    4.0
  6   -16.67928617501       -4.76       -3.55    3.4
  7   -16.67928620247       -7.56       -3.88    2.4
  8   -16.67928622222       -7.70       -4.60    1.0


In [5]:
self_consistent_field(aluminium_setup(4); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32572721537                   -0.56    6.9
  2   -33.33277945969       -2.15       -1.00    1.0
  3   -33.33410853640       -2.88       -1.74    4.8
  4   -33.33469836140       -3.23       -2.35    3.4
  5   -33.33690821334       -2.66       -2.52   14.8
  6   -33.33693554300       -4.56       -2.82    3.4
  7   -33.33694377268       -5.08       -3.68    3.1
  8   -33.33694385938       -7.06       -3.91    5.2
  9   -33.33694392602       -7.18       -4.76    1.5


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298457363999                   -0.85    5.1
  2   -8.300263701514       -2.74       -1.59    1.0
  3   -8.300432114097       -3.77       -2.44    6.0
  4   -8.300324029744   +   -3.97       -2.18    9.1
  5   -8.300464606337       -3.85       -3.88    3.4
  6   -8.300464610764       -8.35       -4.02    2.6


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32578937007                   -0.56    6.8
  2   -33.30071737937   +   -1.60       -1.26    1.0
  3   -1.254386350754   +    1.51       -0.31    8.9
  4   -33.32886098486        1.51       -1.81    7.2
  5   -33.19141225342   +   -0.86       -1.29    5.2
  6   -32.76894561808   +   -0.37       -1.19    4.5
  7   -33.29171718184       -0.28       -1.71    4.5
  8   -33.33627073431       -1.35       -2.22    3.9
  9   -33.33586720095   +   -3.39       -2.22    2.8
 10   -33.33691172460       -2.98       -2.87    1.8
 11   -33.33692536927       -4.87       -3.00    2.2
 12   -33.33693114544       -5.24       -3.09    1.0
 13   -33.33694184949       -4.97       -3.44    2.2
 14   -33.33694325717       -5.85       -3.80    2.2
 15   -33.33694362269       -6.44       -4.19    2.9


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.